## Rahel Ghebrekidan
## Project 3 - Data Wrangling with MongoDB
## Nanodegree Data Analyst 
## Udacity

### Data: July 22, 2016

# Open Street Map - Santa Monica, California

# Parsing XML file 

In [2]:
% cd 'C:\Users\Rahel G G\Udacity\Project_3'

C:\Users\Rahel G G\Udacity\Project_3


In [3]:
#Parsing xml file
import xml.etree.ElementTree as ET
import pprint
import os
SM_data= "Santa_Monica.osm"
def count_tags(filename):
        tags = {}
        for event, elem in ET.iterparse(filename):
            if elem.tag in tags: 
                tags[elem.tag] += 1
            else:
                tags[elem.tag] = 1
        return tags
SM_tags = count_tags(SM_data)
pprint.pprint(SM_tags)

{'bounds': 1,
 'member': 4167,
 'nd': 544106,
 'node': 486074,
 'osm': 1,
 'relation': 792,
 'tag': 320481,
 'way': 51875}


### Volunteers involved in this map editing 

In [4]:
def process_map(filename):
    users = set()
    for _, element in ET.iterparse(filename):
        for e in element:
            if 'uid' in e.attrib:
                users.add(e.attrib['uid'])
    return users
users = process_map(SM_data)
len(users)

351

#### Checking  lowercase(lower-colon), with colon in their names(lower-colon) and tags with problematic characters(Problemchars)


In [5]:
import re

lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')


def key_type(element, keys):
    if element.tag == "tag":
        for tag in element.iter('tag'):
            k = tag.get('k')
            if lower.search(k):
                keys['lower'] += 1
            elif lower_colon.search(k):
                keys['lower_colon'] += 1
            elif problemchars.search(k):
                keys['problemchars'] += 1
            else:
                keys['other'] += 1
    return keys


def process_map(filename):
    keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}
    for _, element in ET.iterparse(filename):
        keys = key_type(element, keys)

    return keys

cal_keys = process_map(SM_data)
pprint.pprint(cal_keys)


{'lower': 184178, 'lower_colon': 134423, 'other': 1879, 'problemchars': 1}



##  Problem Encountering 

I will check if there are problem with Street names(using abbreviations) and with the zip codes.


###  Street address

Below are mentioned street names expected and abbrevations.


In [6]:
from collections import defaultdict

street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE) 

expected = ["Avenue", "Boulevard", "Commons", "Court", "Drive", "Lane", "Parkway","Access","Highway", "Walk", 
                         "Place", "Road", "Square", "Street", "Trail", "Way", "access", 
                          "Center", "access", "1101", "1180","Pico", "North", "Marina","Promenada","Broadway" ]

mapping = {'Ave'  : 'Avenue',
           'Blvd' : 'Boulevard',
           'Blvd.': "Boulevard",
           'Bd.'  : 'Boulevard',
           'Bvd'  : 'Boulevard',
           'Bvd.' : 'Boulevard',
           'Dr'   : 'Drive',
           'Ln'   : 'Lane',
           'Pkwy' : 'Parkway',
           'Rd'   : 'Road',
           'Rd.'  : 'Road',
           'St.'   : 'Street',
           'Ct'   : "Court",
           'Cir'  : "Circle",
           'Cr'   : "Court",
           'ave'  : 'Avenue',
           'Hwg'  : 'Highway',
           'Hwy'  : 'Highway',
           'Sq'   : "Square",
           'St.'  : "Street"}
     

         

#### Matching the street names in the data with the above abbreviations and expected names 


In [7]:
def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)

def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")

def audit(osmfile):
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])

    return street_types

In [8]:
cal_street_types = audit(SM_data)

#### Printing Streets with problem 

In [9]:
pprint.pprint(dict(cal_street_types))

{'Ave': set(['Montana Ave', 'Ohio Ave', 'Olive Ave']),
 'Bd.': set(['Ocean Bd.']),
 'Blvd': set(['Lincoln Blvd',
              'Pico Blvd',
              'Santa Monica Blvd',
              'W Pico Blvd',
              'W Washington Blvd',
              'Washington Blvd',
              'Wilshire Blvd',
              'Wishire Blvd']),
 'Blvd.': set(['Olympic Blvd.']),
 'Bvd': set(['Santa Monica Bvd']),
 'Dr': set(['Entrada Dr']),
 'Ln': set(['Walnut Ln']),
 'Promenade': set(['Third Street Promenade']),
 'South': set(['Donald Douglas Loop South']),
 'St.': set(['Main St.']),
 'ave': set(['Olive ave'])}


#### Replacing the Street names with problem 

In [10]:
def update_street(name, mapping, regex):
    m = regex.search(name)
    if m:
        street_type = m.group()
        if street_type in mapping:
            name = re.sub(regex, mapping[street_type], name)

        return name

for street_type, ways in cal_street_types.iteritems():
    for name in ways:
        better_name = update_street(name, mapping, street_type_re)
        print name, "=>", better_name

Walnut Ln => Walnut Lane
Main St. => Main Street
Ocean Bd. => Ocean Boulevard
Olive Ave => Olive Avenue
Ohio Ave => Ohio Avenue
Montana Ave => Montana Avenue
Third Street Promenade => Third Street Promenade
Santa Monica Bvd => Santa Monica Boulevard
Pico Blvd => Pico Boulevard
Lincoln Blvd => Lincoln Boulevard
Wishire Blvd => Wishire Boulevard
W Washington Blvd => W Washington Boulevard
Wilshire Blvd => Wilshire Boulevard
W Pico Blvd => W Pico Boulevard
Washington Blvd => Washington Boulevard
Santa Monica Blvd => Santa Monica Boulevard
Olive ave => Olive Avenue
Entrada Dr => Entrada Drive
Donald Douglas Loop South => Donald Douglas Loop South
Olympic Blvd. => Olympic Boulevard



####  Zip codes 



In [11]:
from collections import defaultdict
# Zipcodes exceeding or less than 5 digits and with prefix are invalid
def audit_zipcode(invalid_zipcodes, zipcode):
    valid_zipcodes = re.match(r'^[0-9]{5}$', zipcode)
    if not valid_zipcodes:
                   invalid_zipcodes[valid_zipcodes].add(zipcode)
        
def is_zipcode(elem):
    return (elem.attrib['k'] == "addr:postcode")

def audit_zip(osmfile):
    osm_file = open(osmfile, "r")
    invalid_zipcodes = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_zipcode(tag):
                    audit_zipcode(invalid_zipcodes,tag.attrib['v'])

    return invalid_zipcodes

audit_zipcode = audit_zip(SM_data)


In [12]:
pprint.pprint(dict(audit_zipcode))

{None: set(['90025-9998',
            '90064-1508',
            '90272-3719',
            '90291-3879',
            '90401-2405',
            'CA 90272',
            'CA 90291',
            'CA 90401',
            'CA 90404',
            'CA 90405'])}


#### Updating Postcodes 

In [13]:
def update_zipcode(zipcode):
    # Deleting the digits and the hyphen after the fifth digit and the prefix "CA"
    Ext_check = re.findall('(\d{5})-\d{4}', zipcode)
    if Ext_check:
        return re.sub('(\d{5})-\d{4}', '\\1', zipcode) 
    else:
        return re.sub('[^0-9]', '', zipcode)[:5]
for zipcodes, ways in audit_zipcode.iteritems():
    for zipcode in ways:
        better_zipcode = update_zipcode(zipcode)
        print zipcode, "=>", better_zipcode

90025-9998 => 90025
CA 90291 => 90291
90401-2405 => 90401
CA 90272 => 90272
CA 90404 => 90404
90272-3719 => 90272
CA 90401 => 90401
CA 90405 => 90405
90064-1508 => 90064
90291-3879 => 90291


##  Phone Numbers

In [16]:
from collections import defaultdict

def audit_phone_number(invalid_phone_numbers, phone_number):
    #telephone numbers with more than 11 digits and separated by dot are considered invalid
    valid_phone_numbers = re.match(r'^(?:(?:\+?1\s*(?:[-]\s*)?)?(?:\(\s*([2-9]1[02-9]|[2-9][02-8]1|[2-9][02-8][02-9])\s*\)|([2-9]1[02-9]|[2-9][02-8]1|[2-9][02-8][02-9]))\s*(?:[-]\s*)?)?([2-9]1[02-9]|[2-9][02-9]1|[2-9][02-9]{2})\s*(?:[-]\s*)?([0-9]{4})(?:\s*(?:#|x\.?|ext\.?|extension)\s*(\d+))?$', phone_number)
  
    if not valid_phone_numbers:
                   invalid_phone_numbers[valid_phone_numbers].add(phone_number)
        
def is_phone_number(elem):
    return (elem.attrib['k'] == "phone")

def audit_phone_num(osmfile):
    osm_file = open(osmfile, "r")
    invalid_phone_numbers = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_phone_number(tag):
                    audit_phone_number(invalid_phone_numbers,tag.attrib['v'])

    return invalid_phone_numbers

audit_phone_number = audit_phone_num(SM_data)

In [17]:
pprint.pprint(dict(audit_phone_number))

{None: set(['+01-310-260-6308',
            '310. 444. 0045',
            '310.473.1447',
            '310.478.3545',
            '310.826.2229'])}


In [18]:
def update_phone(phone_number):
    valid_phone = re.sub('[^0-9]+', '', phone_number)
    phone_pattern = re.sub("(\d)(?=(\d{3})+(?!\d))", r"\1-", "%d" % int(valid_phone[:-1])) + valid_phone[-1]
    return phone_pattern
for phone_number, ways in audit_phone_number.iteritems():
    for phone in ways:
        better_phone_num = update_phone(phone)
        print phone, "=>", better_phone_num

+01-310-260-6308 => 1-310-260-6308
310. 444. 0045 => 310-444-0045
310.473.1447 => 310-473-1447
310.478.3545 => 310-478-3545
310.826.2229 => 310-826-2229


## Converting XML to JSON format

Inorder to be readable on Mongodb, the XML formate should be changed to JSON format


In [19]:
import re
import codecs 
import json

lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

CREATED = [ "version", "changeset", "timestamp", "user", "uid"]


def shape_element(element):
    node = {}
    # processing nodes and ways
    if element.tag == "node" or element.tag == "way" :
        for key in element.attrib.keys():
            val = element.attrib[key]
            node["type"] = element.tag
            if key in CREATED:
                if not "created" in node.keys():
                    node["created"] = {}
                node["created"][key] = val
            elif key == "lat" or key == "lon":
                if not "pos" in node.keys():
                    node["pos"] = [0.0, 0.0]
                old_pos = node["pos"]
                if key == "lat":
                    new_pos = [float(val), old_pos[1]]
                else:
                    new_pos = [old_pos[0], float(val)]
                node["pos"] = new_pos
            else:
                node[key] = val
        #itering child element
        for child in element. iter("tag"):
            if not problemchars.search(child.attrib['k']):
                if lower_colon.search(child.attrib['k']):
                    if child.attrib['k'].find('addr') == 0:
                    
                        if 'address' not in node:
                            node['address'] = {}
                        sub_attr = child.attrib['k'].split(':')
                        node['address'][sub_attr[0]] = child.attrib['v']
                        address = node['address']
                        sub_key = sub_attr[1]
                        #updating street
                        if sub_key == 'street':
                            address['street'] = update_street(child.attrib['v'], mapping, street_type_re)
                        #updating zipcode
                        elif sub_key == 'postcode':
                            address['postcode'] = update_zipcode(child.attrib['v'])
                        else:
                            address[sub_key] = child.attrib['v']
                    else:
                        node[child.attrib['k']] = child.attrib['v']
                elif child.attrib['k'].find(':') == -1:
                    node[child.attrib['k']] = child.attrib['v']
                    key_itm = child.attrib['k']
                    #updating telephone number
                    if key_itm == 'phone':
                        node['phone'] = update_phone(child.attrib['v'])
                
        for nd in element.iter("nd"):
            if 'node_refs' not in node:
                node['node_refs'] = []
            node['node_refs'].append(nd.attrib['ref'])

        return node
    else:
        return None


def process_map(file_in, pretty = False):
    file_out = "{0}.json".format(file_in)
    data = []
    with codecs.open(file_out, "w") as fo:
        for _, element in ET.iterparse(file_in):
            el = shape_element(element)
            if el:
                data.append(el)
                if pretty:
                    fo.write(json.dumps(el, indent=2)+"\n")
                else:
                    fo.write(json.dumps(el) + "\n")
    return data
process_map(SM_data, False)


[{'created': {'changeset': '11239147',
   'timestamp': '2012-04-09T15:23:25Z',
   'uid': '104962',
   'user': 'techlady',
   'version': '21'},
  'id': '14752647',
  'pos': [34.0278652, -118.4512786],
  'type': 'node'},
 {'created': {'changeset': '11245718',
   'timestamp': '2012-04-09T22:09:46Z',
   'uid': '104962',
   'user': 'techlady',
   'version': '20'},
  'id': '14752649',
  'pos': [34.0230505, -118.4746697],
  'type': 'node'},
 {'created': {'changeset': '3784290',
   'timestamp': '2010-02-03T16:55:09Z',
   'uid': '104962',
   'user': 'techlady',
   'version': '3'},
  'id': '15857078',
  'pos': [34.0288171, -118.4506263],
  'type': 'node'},
 {'created': {'changeset': '3784290',
   'timestamp': '2010-02-03T16:55:09Z',
   'uid': '104962',
   'user': 'techlady',
   'version': '3'},
  'id': '15857079',
  'pos': [34.0289905, -118.4503527],
  'type': 'node'},
 {'created': {'changeset': '10561877',
   'timestamp': '2012-02-01T19:53:49Z',
   'uid': '207745',
   'user': 'NE2',
   'version

In [20]:
data = process_map(SM_data)

## Opening the Dataset in MongoDB

In [21]:
from pymongo import MongoClient

In [22]:
client  = MongoClient('mongodb://localhost:27017')
db = client.examples

In [23]:
[db.St_Monica_CA.insert(e) for e in data]

C:\Users\Rahel G G\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  if __name__ == '__main__':


[ObjectId('57929ed83c97f7303c323998'),
 ObjectId('57929ed83c97f7303c323999'),
 ObjectId('57929ed83c97f7303c32399a'),
 ObjectId('57929ed83c97f7303c32399b'),
 ObjectId('57929ed83c97f7303c32399c'),
 ObjectId('57929ed83c97f7303c32399d'),
 ObjectId('57929ed83c97f7303c32399e'),
 ObjectId('57929ed83c97f7303c32399f'),
 ObjectId('57929ed83c97f7303c3239a0'),
 ObjectId('57929ed83c97f7303c3239a1'),
 ObjectId('57929ed83c97f7303c3239a2'),
 ObjectId('57929ed83c97f7303c3239a3'),
 ObjectId('57929ed83c97f7303c3239a4'),
 ObjectId('57929ed83c97f7303c3239a5'),
 ObjectId('57929ed83c97f7303c3239a6'),
 ObjectId('57929ed83c97f7303c3239a7'),
 ObjectId('57929ed83c97f7303c3239a8'),
 ObjectId('57929ed83c97f7303c3239a9'),
 ObjectId('57929ed83c97f7303c3239aa'),
 ObjectId('57929ed83c97f7303c3239ab'),
 ObjectId('57929ed83c97f7303c3239ac'),
 ObjectId('57929ed83c97f7303c3239ad'),
 ObjectId('57929ed83c97f7303c3239ae'),
 ObjectId('57929ed83c97f7303c3239af'),
 ObjectId('57929ed83c97f7303c3239b0'),
 ObjectId('57929ed83c97f7

## City names in the Dataset

### Checking if the city name is correct in the dataset

In [36]:
city = db.St_Monica_CA.aggregate( [ 
    { "$match" : { "address.city" : { "$exists" : 1} } }, 
    { "$group" : { "_id" : "$address.city", "count" : { "$sum" : 1} } },  
    { "$sort" : { "count" : -1}},
      ])
print(list(city))

[{u'count': 92, u'_id': u'Santa Monica'}, {u'count': 34, u'_id': u'Los Angeles'}, {u'count': 19, u'_id': u'Marina Del Rey CA'}, {u'count': 13, u'_id': u'Marina Del Rey'}, {u'count': 10, u'_id': u'Venice'}, {u'count': 7, u'_id': u'Marina del Rey'}, {u'count': 3, u'_id': u'Los Angeles-Venice'}, {u'count': 2, u'_id': u'West Los Angeles'}, {u'count': 1, u'_id': u'Venice CA'}, {u'count': 1, u'_id': u'Marina del Ray'}, {u'count': 1, u'_id': u'santa Monica'}, {u'count': 1, u'_id': u'Pacific Palisades'}]


# Updating City Names

In [24]:
db.St_Monica_CA.update_many(
    {"address.city": {"$in": [
        "santa Monica",
        "Los Angeles",
        "West Los Angeles" 
    ]}}, 
    {"$set": {"address.city": "Santa Monica"}}, 
    upsert = True
)

In [25]:
db.St_Monica_CA.update_many(
    {"address.city": {"$in": [
        "Marina Del Rey CA",
        "Marina del Rey ",
        "Marina del Ray" 
    ]}}, 
    {"$set": {"address.city": "Santa Monica"}}, 
    upsert = True
)

In [27]:
db.St_Monica_CA.update_many(
    {"address.city": {"$in": [
        "Los Angeles-Venice",
        "Venice CA "
         ]}}, 
    {"$set": {"address.city": "Santa Monica"}}, 
    upsert = True
)

## Checking if city name is updated

In [46]:
city_updated = db.St_Monica_CA.aggregate( [ 
    { "$match" : { "address.city" : { "$exists" : 1} } }, 
    { "$group" : { "_id" : "$address.city", "count" : { "$sum" : 1} } },  
    { "$sort" : { "count" : -1}},
      ])
print(list(city_updated))

[{u'count': 129, u'_id': u'Santa Monica'}, {u'count': 40, u'_id': u'Marina Del Rey'}, {u'count': 14, u'_id': u'Venice'}, {u'count': 1, u'_id': u'Pacific Palisades'}]


## Analyzing Dataset 

#### The data size in OSM and Json Format

In [48]:
import os
print 'Size of the OSM file = {} MB'.format(os.path.getsize(SM_data)/1.0e6) 
print 'Size of JSON file = {} MB'.format(os.path.getsize(SM_data + ".json")/1.0e6)

Size of the OSM file = 111.134183 MB
Size of JSON file = 125.765236 MB


## Number of Documents

In [35]:
db.St_Monica_CA.find().count()

537949

## Number of Nodes

In [49]:
db.St_Monica_CA.find({"type":"node"}).count()

486071

In [50]:
db.St_Monica_CA.find({"type":"way"}).count()

51872

## Number of unique users

In [51]:
len(db.St_Monica_CA.distinct('created.user'))

341


## Amenities in Santa Monica

In [61]:
amenity = db.St_Monica_CA.aggregate([{'$match': {'amenity': {'$exists': 1}}}, \
                                {'$group': {'_id': '$amenity', \
                                            'count': {'$sum': 1}}}, \
                                {'$sort': {'count': -1}}, \
                                {'$limit': 10}])
print(list(amenity))

[{u'count': 176, u'_id': u'parking'}, {u'count': 86, u'_id': u'restaurant'}, {u'count': 83, u'_id': u'bicycle_rental'}, {u'count': 57, u'_id': u'school'}, {u'count': 50, u'_id': u'place_of_worship'}, {u'count': 42, u'_id': u'cafe'}, {u'count': 41, u'_id': u'drinking_water'}, {u'count': 23, u'_id': u'fast_food'}, {u'count': 20, u'_id': u'toilets'}, {u'count': 17, u'_id': u'fuel'}]


## Types of Cuisines 

In [55]:
cuisine = db.St_Monica_CA.aggregate([{"$match":{"amenity":{"$exists":1},
                                 "amenity":"restaurant",}},      
                      {"$group":{"_id":{"Food":"$cuisine"},
                                 "count":{"$sum":1}}},
                      {"$project":{"_id":0,
                                  "Food":"$_id.Food",
                                  "Count":"$count"}},
                      {"$sort":{"Count":-1}}, 
                      ])
print(list(cuisine))

[{u'Food': None, u'Count': 43}, {u'Food': u'american', u'Count': 9}, {u'Food': u'mexican', u'Count': 5}, {u'Food': u'italian', u'Count': 5}, {u'Food': u'burger', u'Count': 4}, {u'Food': u'regional', u'Count': 2}, {u'Food': u'thai', u'Count': 2}, {u'Food': u'steakhouse,_italian,_mediterranean', u'Count': 2}, {u'Food': u'sandwich', u'Count': 2}, {u'Food': u'pizza', u'Count': 2}, {u'Food': u'indian', u'Count': 1}, {u'Food': u'french', u'Count': 1}, {u'Food': u'vegan', u'Count': 1}, {u'Food': u'greek', u'Count': 1}, {u'Food': u'argentinian', u'Count': 1}, {u'Food': u'vegetarian', u'Count': 1}, {u'Food': u'international', u'Count': 1}, {u'Food': u'seafood', u'Count': 1}, {u'Food': u'thai;chinese', u'Count': 1}, {u'Food': u'asian', u'Count': 1}]


## Tourism  Sites

In [62]:
Tourism_Sites = db.St_Monica_CA.aggregate( [ 
    { "$match" : { "tourism" : { "$exists" : 1} } }, 
    { "$group" : { "_id" : "$tourism", "count" : { "$sum" : 1} } },  
    { "$sort" : { "count" : -1}},
      ])
print(list(Tourism_Sites))

[{u'count': 54, u'_id': u'hotel'}, {u'count': 12, u'_id': u'attraction'}, {u'count': 9, u'_id': u'motel'}, {u'count': 4, u'_id': u'museum'}, {u'count': 3, u'_id': u'picnic_site'}, {u'count': 3, u'_id': u'hostel'}, {u'count': 2, u'_id': u'artwork'}, {u'count': 1, u'_id': u'theme_park'}, {u'count': 1, u'_id': u'viewpoint'}, {u'count': 1, u'_id': u'information'}, {u'count': 1, u'_id': u'apine_hut'}]


## Reference:

1.	Data design Goup.(2014).JSON : Conver XML to JSON: Retrieved from http://convertjson.com/xml-to-json.htm
2.	Sudirman,J.(2013). Open Street Map: Retrieved from http://napitupulu-jon.appspot.com/posts/wrangling-openstreetmap.html
3.	Allanbreyes.(2015). Udacity Data Science:P2: Retrieved from  https://github.com/allanbreyes/udacity-data-science/tree/master/p2
4.	Bogotobogo.(2016). MongoDB with Pymongo I: Retrieved from http://www.bogotobogo.com/python/MongoDB_PyMongo/python_MongoDB_pyMongo_tutorial_installing.php
5.	Stack over flow: Retrieved from http://stackoverflow.com/questions/2577236/regex-for-zip-code
6.	Regular expression101: regular expression tester : Retrieved from https://regex101.com/r/lV5yU9/1
7.	 MongoDB (2008). MongoDB. Retrieved from:
  https://docs.mongodb.com/manual/reference/command/getLastError/  
